<a href="https://colab.research.google.com/github/goshan16389/ii_ubiet_mir/blob/main/5task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка библиотек для прогнозирования временных рядов
!pip install u8darts -q
!pip install --upgrade pytorch-forecasting pytorch-lightning

In [ ]:
# Импорт основных библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import io
import requests

from darts import TimeSeries
from darts.models import NBEATSModel
from darts.metrics import mape, smape as darts_smape
from darts.utils.missing_values import fill_missing_values

from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# Загрузка и чтение датасета FAOSTAT (производство сельхозкультур)
url = "https://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_Livestock_E_All_Data_(Normalized).zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
csv_name = z.namelist()[0]
df = pd.read_csv(z.open(csv_name), encoding="latin1")

# Фильтрация интересующих культур
items = [
    "Apples", "Cabbages", "Cherries", "Cocoa beans",
    "Cranberries", "Cucumbers and gherkins", "Grapes",
    "Maize (corn)", "Oranges", "Tomatoes", "Wheat"
]

df = df[(df["Item"].isin(items)) & (df["Element"] == "Production")]

# Агрегация данных по годам и приведение к временному формату
df = df.groupby(["Item", "Year"])["Value"].sum().reset_index()
df.rename(columns={"Item": "product", "Year": "year", "Value": "y"}, inplace=True)
df["ds"] = pd.to_datetime(df["year"], format="%Y")
df = df.sort_values(["product", "ds"]).reset_index(drop=True)

In [ ]:
df.head(5)

In [ ]:
HORIZON = 3          # сколько лет прогнозируем
HISTORY = 18  # сколько лет считываем для прогноза

In [ ]:
# Формирование временных рядов для каждого продукта
series_dict = {}
products = df["product"].unique()

for product in products:
    product_df = df[df["product"] == product][["ds", "y"]].set_index("ds")
    ts = TimeSeries.from_series(product_df["y"])
    ts = fill_missing_values(ts, method="linear")
    series_dict[product] = ts

# Разделение временных рядов на обучающую и валидационную части
train_series = {}
val_series = {}

for product, ts in series_dict.items():
    train_series[product] = ts[:-HORIZON]
    val_series[product] = ts[-HORIZON:]

In [ ]:
# Инициализация модели N-BEATS
nbeats_model = NBEATSModel(
    input_chunk_length=HISTORY,
    output_chunk_length=HORIZON,
    generic_architecture=True,
    num_stacks=30,
    num_blocks=1,
    num_layers=4,
    layer_widths=512,
    n_epochs=20,
    batch_size=16,
    random_state=42,
    pl_trainer_kwargs={
        "enable_progress_bar": False,
        "enable_model_summary": False,
        "logger": False
    }
)

# Обучение модели N-BEATS на всех временных рядах
nbeats_model.fit(series=list(train_series.values()), verbose=True)

In [ ]:
# Сохранение обученной модели
nbeats_model.save("nbeats_model_trained.darts")

In [ ]:
# Функция расчёта MAPE и SMAPE
def compute_metrics(actual, pred):
    actual, pred = np.asarray(actual).flatten(), np.asarray(pred).flatten()
    if len(actual) != len(pred) or not (mask := actual != 0).any():
        return np.nan, np.nan
    abs_diff = np.abs(pred[mask] - actual[mask])
    mape = np.mean(abs_diff / np.abs(actual[mask])) * 100
    smape = 200 * np.mean(abs_diff / (np.abs(actual[mask]) + np.abs(pred[mask]) + 1e-10))
    return mape, smape

# Сравнение моделей ARIMA и N-BEATS по метрикам качества
results = []
for product in train_series:
    actual = val_series[product].values().flatten()
    train = train_series[product].values().flatten()

    arima_mape, arima_smape = np.nan, np.nan
    try:
        arima_pred = ARIMA(train, order=(1,1,1)).fit().forecast(steps=HORIZON)
        arima_mape, arima_smape = compute_metrics(actual, arima_pred)
    except:
        pass

    nbeats_pred = nbeats_model.predict(n=HORIZON, series=train_series[product]).values().flatten()
    nbeats_mape, nbeats_smape = compute_metrics(actual, nbeats_pred)

    results.append({
        "product": product,
        "ARIMA_MAPE": arima_mape,
        "ARIMA_SMAPE": arima_smape,
        "NBEATS_MAPE": nbeats_mape,
        "NBEATS_SMAPE": nbeats_smape
    })

# Вывод и сохранение сравнительной таблицы метрик
print("\nСравнительная таблица:")
print(pd.DataFrame(results).sort_values("NBEATS_MAPE").to_string(index=False))

with open('model_report.txt', 'w', encoding='utf-8') as f:
    f.write(pd.DataFrame(results).sort_values("NBEATS_MAPE").to_string(index=False))

In [ ]:
# Параметр начала отображаемого временного интервала
START_YEAR = 2000

# Визуализация прогнозов для каждого продукта
for product in train_series:
    train_ts = train_series[product]
    actual_ts = val_series[product]

    # Прогноз N-BEATS
    nbeats_pred = nbeats_model.predict(n=HORIZON, series=train_ts)

    # Прогноз ARIMA
    arima_pred_ts = None
    try:
        arima_fit = ARIMA(train_ts.values().flatten(), order=(1,1,1)).fit()
        forecast = arima_fit.forecast(steps=HORIZON)
        arima_pred_ts = TimeSeries.from_times_and_values(
            actual_ts.time_index,
            forecast,
            columns=[f"ARIMA_{product}"]
        )
    except Exception as e:
        print(f"ARIMA failed for {product}: {e}")

    plt.figure(figsize=(14, 6))

    train_ts[START_YEAR-1961:].plot(label="Исторические данные", color="gray", alpha=0.7, lw=2)
    actual_ts.plot(label="Фактические значения", color="royalblue", marker="o", ms=4, lw=2)
    nbeats_pred.plot(label="N-BEATS прогноз", color="darkred", ls="--", marker="o", ms=4, lw=2)

    if arima_pred_ts is not None:
        arima_pred_ts.plot(label="ARIMA прогноз", color="darkgreen", ls=":", marker="x", ms=4, lw=2)

    plt.axvline(train_ts.time_index[-1], color="black", ls=":", lw=1.5, label="Начало прогноза")

    plt.xlim(train_ts[START_YEAR-1961:].time_index[0], actual_ts.time_index[-1])
    plt.title(f"{product} — Прогноз на {HORIZON} года")
    plt.xlabel("Год")
    plt.ylabel("Производство")
    plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    safe_name = product.replace(" ", "_").replace("(", "").replace(")", "")
    plt.savefig(f"{safe_name}_graphics.png", dpi=150, bbox_inches="tight")
    plt.show()

In [ ]:
from google.colab import files

files.download('/content/nbeats_model_trained.darts')
files.download('model_report.txt')
files.download('/content/nbeats_model_trained.darts.ckpt')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
nbeats_model = NBEATSModel.load("nbeats_model_trained.darts")